# Deliverable 1: Retrieve Weather Data

In [1]:
# Dependencies and Setup
import numpy as np 
from citipy import citipy
import pandas as pd
import requests
import gmaps
import json

# Import API key
from config import g_key
from config import weather_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [3]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

20.149805342802438 97.1271347633392
-8.216467310445893 146.51328685878514
-37.353387887586976 -81.82432715837302
29.37631579337959 101.95712023184501
47.17295472813612 -104.3908126979763
17.806545886017346 -24.885151612204538
64.84707538549375 135.70615732230902
31.25602521100184 168.83634738146674
-75.1766933421107 109.26308750757033
76.3951240747522 112.34424544979424
-8.625094175858095 -179.0309654073627
-83.97090967327483 26.72581523957666
-58.00605301677972 -133.33480407061353
-5.191957024692584 -71.53063543367206
-31.170061583575063 63.22127254021004
-8.96859133194691 51.81785966437934
47.55310625427208 30.259775075997652
-37.460130216890434 84.97097021727569
-47.87621117953274 -170.43987099394593
-48.50062693044725 50.9812433192393
48.27223244841554 -49.51817992219256
22.923875620401745 149.66075164271518
-24.767371273914875 -118.25026639535551
-4.434407306231293 -128.55472223879235
-29.52489382594753 -97.90126798159253
-79.28141588079639 74.84737454602728
-15.263290624794834 15

-89.04450892114933 -123.7211531585086
1.1915777444953477 92.47211319376436
-34.332343197961585 127.20963406970918
20.369916428985192 3.1923759955154196
8.32293864265786 -151.18157311269562
82.39631472196001 30.314491356468892
72.12810759007604 -147.08743838436286
-23.78722321434438 90.67243708201113
46.31158597030651 85.50246270864812
81.99820053081086 69.53592978003903
17.330541295748688 171.26719979866112
-49.01001180723852 -66.06779235557492
9.55054630848079 -128.42555672299312
36.77988092211669 -178.1565253784545
-31.99917833970676 166.4853940396692
-77.80804233107007 -149.57587850279492
-31.841402747228393 -18.14377225336517
-13.944383766215168 34.102342084255866
29.92772094377831 178.5567428348591
30.77950348766572 -17.391260561643804
-80.31959494044348 -88.40045305556602
28.469912181180064 51.07264897303074
52.81514031841294 68.88798970799425
-12.887107070857567 15.061702327304232
-51.93272582607597 -115.56772659463869
36.76432899071479 43.87421956928472
63.16687692248033 128.45

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

785

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=363f097f19cb97b414c974ec3166c407


In [8]:
# Retrieve the following information from the API call:
# Latitude and longitude
# Maximum temperature
# Percent humidity
# Percent cloudiness
# Wind speed
# Weather description (for example, clouds, fog, light rain, clear sky)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | loikaw
Processing Record 2 of Set 1 | kerema
Processing Record 3 of Set 1 | lebu
Processing Record 4 of Set 1 | yaan
Processing Record 5 of Set 1 | glendive
Processing Record 6 of Set 1 | ponta do sol
Processing Record 7 of Set 1 | khandyga
Processing Record 8 of Set 1 | severo-kurilsk
Processing Record 9 of Set 1 | albany
Processing Record 10 of Set 1 | saskylakh
Processing Record 11 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 12 of Set 1 | bredasdorp
Processing Record 13 of Set 1 | rikitea
Processing Record 14 of Set 1 | benjamin constant
Processing Record 15 of Set 1 | souillac
Processing Record 16 of Set 1 | ambilobe
Processing Record 17 of Set 1 | shyryayeve
Processing Record 18 of Set 1 | busselton
Processing Record 19 of Set 1 | vaini
Processing Record 20 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 21 of Set 1 | torbay
Processing Record 22 of Set

Processing Record 41 of Set 4 | itaituba
Processing Record 42 of Set 4 | japura
Processing Record 43 of Set 4 | nuuk
Processing Record 44 of Set 4 | clyde river
Processing Record 45 of Set 4 | lodwar
Processing Record 46 of Set 4 | lar gerd
City not found. Skipping...
Processing Record 47 of Set 4 | harper
Processing Record 48 of Set 4 | hovd
Processing Record 49 of Set 4 | tasiilaq
Processing Record 50 of Set 4 | vanavara
Processing Record 1 of Set 5 | kudahuvadhoo
Processing Record 2 of Set 5 | palauig
Processing Record 3 of Set 5 | zhuhai
Processing Record 4 of Set 5 | xining
Processing Record 5 of Set 5 | vestmannaeyjar
Processing Record 6 of Set 5 | huarmey
Processing Record 7 of Set 5 | necochea
Processing Record 8 of Set 5 | kamenskoye
City not found. Skipping...
Processing Record 9 of Set 5 | pangody
Processing Record 10 of Set 5 | pilao arcado
City not found. Skipping...
Processing Record 11 of Set 5 | komsomolskiy
Processing Record 12 of Set 5 | raudeberg
Processing Record 13

Processing Record 28 of Set 8 | hami
Processing Record 29 of Set 8 | kadykchan
City not found. Skipping...
Processing Record 30 of Set 8 | roma
Processing Record 31 of Set 8 | matara
Processing Record 32 of Set 8 | caxito
Processing Record 33 of Set 8 | maneadero
Processing Record 34 of Set 8 | kirakira
Processing Record 35 of Set 8 | pisco
Processing Record 36 of Set 8 | rajapur
Processing Record 37 of Set 8 | hihifo
City not found. Skipping...
Processing Record 38 of Set 8 | boundiali
Processing Record 39 of Set 8 | kaitangata
Processing Record 40 of Set 8 | isangel
Processing Record 41 of Set 8 | tanshui
City not found. Skipping...
Processing Record 42 of Set 8 | nizhneyansk
City not found. Skipping...
Processing Record 43 of Set 8 | fukuma
Processing Record 44 of Set 8 | kualakapuas
Processing Record 45 of Set 8 | clonakilty
Processing Record 46 of Set 8 | uyuni
Processing Record 47 of Set 8 | springbok
Processing Record 48 of Set 8 | martapura
Processing Record 49 of Set 8 | touro

Processing Record 12 of Set 12 | khormuj
City not found. Skipping...
Processing Record 13 of Set 12 | longonjo
Processing Record 14 of Set 12 | irbil
Processing Record 15 of Set 12 | sangar
Processing Record 16 of Set 12 | imuris
Processing Record 17 of Set 12 | constitucion
Processing Record 18 of Set 12 | cap-chat
Processing Record 19 of Set 12 | waingapu
Processing Record 20 of Set 12 | bilibino
Processing Record 21 of Set 12 | dwarka
Processing Record 22 of Set 12 | mizdah
Processing Record 23 of Set 12 | sur
Processing Record 24 of Set 12 | nampa
Processing Record 25 of Set 12 | otradnoye
Processing Record 26 of Set 12 | port lincoln
Processing Record 27 of Set 12 | cordoba
Processing Record 28 of Set 12 | atambua
Processing Record 29 of Set 12 | marawi
Processing Record 30 of Set 12 | tricase
Processing Record 31 of Set 12 | san policarpo
Processing Record 32 of Set 12 | sarai
Processing Record 33 of Set 12 | salvador
Processing Record 34 of Set 12 | tocantinopolis
City not found

Processing Record 47 of Set 15 | vostok
Processing Record 48 of Set 15 | carutapera
Processing Record 49 of Set 15 | cabedelo
Processing Record 50 of Set 15 | aberdeen
Processing Record 1 of Set 16 | elk city
Processing Record 2 of Set 16 | saint-ambroise
Processing Record 3 of Set 16 | mittersill
Processing Record 4 of Set 16 | monteagudo
Processing Record 5 of Set 16 | kollumerland
City not found. Skipping...
Processing Record 6 of Set 16 | boquira
Processing Record 7 of Set 16 | along
Processing Record 8 of Set 16 | shaowu
Processing Record 9 of Set 16 | neuquen
Processing Record 10 of Set 16 | mafeteng
Processing Record 11 of Set 16 | martinho campos
Processing Record 12 of Set 16 | vallenar
Processing Record 13 of Set 16 | beloha
Processing Record 14 of Set 16 | jimo
Processing Record 15 of Set 16 | korla
Processing Record 16 of Set 16 | gouyave
Processing Record 17 of Set 16 | woodland
Processing Record 18 of Set 16 | kulhudhuffushi
Processing Record 19 of Set 16 | muika
City not

In [9]:
# Number of cities in the city data array of dictionaries
len(city_data)

710

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Loikaw,MM,19.6742,97.2094,86.05,58,58,2.06,broken clouds
1,Kerema,PG,-7.9631,145.7785,80.64,82,66,5.37,broken clouds
2,Lebu,CL,-37.6167,-73.6500,49.80,87,54,12.12,broken clouds
3,Yaan,NG,7.3833,8.5667,71.49,97,100,5.39,overcast clouds
4,Glendive,US,47.1053,-104.7125,67.62,59,20,6.91,few clouds
5,Ponta Do Sol,PT,32.6667,-17.1000,72.23,69,0,5.41,clear sky
6,Khandyga,RU,62.6667,135.6000,82.83,43,47,4.09,scattered clouds
7,Severo-Kurilsk,RU,50.6789,156.1250,55.00,82,95,5.19,overcast clouds
8,Albany,US,42.6001,-73.9662,62.53,83,1,3.62,clear sky
9,Saskylakh,RU,71.9167,114.0833,82.80,41,29,12.24,scattered clouds


In [13]:
# Create CSV file of DF
output_weather_file = "weather_database/WeatherPy_Database.csv"
# Export CSV file 
city_data_df.to_csv(output_weather_file, index_label="City_ID")